<a href="https://colab.research.google.com/github/VieiraEduardo/ProjetoPLN/blob/main/5_Processamento_de_linguagem_natural_(PLN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Processamento de Linguagem Natural

*   Este exemplo irei mostrar como usar técnicas de NLP para análisar textos, extrair insights, classificar sentimentos e transoformar e transformar textos não estruturados em dados estruturados úteis. Essas técnicas podem ser aplicadas a diversos cados de uso, como análise de sentimentos de produtos, feedback de clientes e muito mais.





1.Preparar e carregar os dados

irei usar um conjunto de dados fictícios de análise de produtos.

In [1]:
import pandas as pd
#Criação de um DataFrame fictício com análises de produtos
data= {
    "review": [
        "This product is great! I love it",
        "Terrible customer service. I am very disappointed.",
        "The quality is fantastic",
        "Not worth the money. Very poor quality",
        "I am extremely satisfied with this purchase."

    ]
}
df=pd.DataFrame(data)
print("dados carregados:")
print(df)



dados carregados:
                                              review
0                   This product is great! I love it
1  Terrible customer service. I am very disappoin...
2                           The quality is fantastic
3             Not worth the money. Very poor quality
4       I am extremely satisfied with this purchase.


2. Pré-processar os textos


*   Limpando os textos convertendo para minúsculos, removendo pontuações, tokennizando, removendo stopwords e lematizando.




In [2]:
import nltk #é uma das bibliotecas mais populares e poderosas para NLP em Python
from nltk.corpus import stopwords #módulo do nltk que contém uma lista de palaras comuns que geralmente são removidas durante o pré processamento de textos em tarefas
from nltk.tokenize import word_tokenize #módulo do nltk que é usada para dividir um texto em palavras individuais ( tokens )
from nltk.stem import WordNetLemmatizer #módulo do nltk que é usada para reduzir palavras a sua forma base ou raiz ( lemmas )
import string #contém diversas constantes e clsses úteis para manipulação de strings

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_text(text):
  #Converter o texto para minúsculos
  text=text.lower()
  #Remover pontuações
  text=text.translate(str.maketrans('', '', string.punctuation))
  #Tokenizar o texto
  tokens=word_tokenize(text)
  #Remover stopwords
  text=[word for word in tokens if word not in stopwords.words('english')]
  #Lematizar as palavras
  lemmatizer=WordNetLemmatizer()
  text=[lemmatizer.lemmatize(word) for word in text]
  #juntar as palavras de volta a uma string
  text=' '.join(text)
  return text
#Aplicar a função de pré processamento
df["cleaned_review"]=df["review"].apply(preprocess_text)
print("\nTextos pré-processdos:")
print(df["cleaned_review"])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...



Textos pré-processdos:
0                        product great love
1    terrible customer service disappointed
2                         quality fantastic
3                  worth money poor quality
4              extremely satisfied purchase
Name: cleaned_review, dtype: object


3.Classificar Sentimentos

In [3]:
from textblob import TextBlob #fornece uma API simples para tarefas  comuns de NLP, como análise de sentimentos, tradução, correção gramatical, tokenização e muito mais
def classify_sentiment(text):
    analysis=TextBlob(text)
    #Classificar como positivo, negativo ou neutro
    if analysis.sentiment.polarity > 0:
        return "positive"
    elif analysis.sentiment.polarity < 0:
        return "negative"
    else:
        return "neutral"

#Aplicar a função de classificação de sentimento
df["sentiment"]=df["cleaned_review"].apply(classify_sentiment)
print("\nClassificação de sentimentos:")
print(df[["review", "sentiment"]])



Classificação de sentimentos:
                                              review sentiment
0                   This product is great! I love it  positive
1  Terrible customer service. I am very disappoin...  negative
2                           The quality is fantastic  positive
3             Not worth the money. Very poor quality  negative
4       I am extremely satisfied with this purchase.  positive


4. Extrair Insigths

In [4]:
from collections import Counter #Módulo de python que fornece ferramentas para trabalhar com coleções
#Contagem das palavras
all_words=' '.join(df["cleaned_review"])
word_freq=Counter(all_words.split())
print("\nPalavras mais frequentes:")
print(word_freq.most_common(10))

#Contagem de sentimentos
sentiment_counts=df["sentiment"].value_counts()
print("\nContagem de sentimentos:")
print(sentiment_counts)



Palavras mais frequentes:
[('quality', 2), ('product', 1), ('great', 1), ('love', 1), ('terrible', 1), ('customer', 1), ('service', 1), ('disappointed', 1), ('fantastic', 1), ('worth', 1)]

Contagem de sentimentos:
sentiment
positive    3
negative    2
Name: count, dtype: int64


5. Transformar textos em dados estruturados



In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer # módulo do scikit-learn que é usado para extrair recursos de texto
# Transformar textos em uma representação vetorial TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['cleaned_review'])

# Mostrar a matriz TF-IDF
tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
print("\nMatriz TF-IDF:")
print(tfidf_df)



Matriz TF-IDF:
   customer  disappointed  extremely  fantastic    great     love     money  \
0       0.0           0.0    0.00000   0.000000  0.57735  0.57735  0.000000   
1       0.5           0.5    0.00000   0.000000  0.00000  0.00000  0.000000   
2       0.0           0.0    0.00000   0.778283  0.00000  0.00000  0.000000   
3       0.0           0.0    0.00000   0.000000  0.00000  0.00000  0.523358   
4       0.0           0.0    0.57735   0.000000  0.00000  0.00000  0.000000   

       poor  product  purchase   quality  satisfied  service  terrible  \
0  0.000000  0.57735   0.00000  0.000000    0.00000      0.0       0.0   
1  0.000000  0.00000   0.00000  0.000000    0.00000      0.5       0.5   
2  0.000000  0.00000   0.00000  0.627914    0.00000      0.0       0.0   
3  0.523358  0.00000   0.00000  0.422242    0.00000      0.0       0.0   
4  0.000000  0.00000   0.57735  0.000000    0.57735      0.0       0.0   

      worth  
0  0.000000  
1  0.000000  
2  0.000000  
3  0.523

In [6]:

from google.colab import files
# Salvar o DataFrame como um arquivo CSV
tfidf_df.to_csv('matriz-tf-idf.csv', index=True)

In [35]:

# Baixar o arquivo CSV para pasta local
files.download('matriz-tf-idf.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>